### 1.schedule库的基本用法

In [ ]:
#schedule库的使用：每两秒执行一次job
import schedule
import time

def job():
    print("i'm working...")

schedule.every(2).seconds.do(job)

while True:
    schedule.run_pending()
    time.sleep(1)

### 2.代码优化
    1.加入try...except保证程序正常运行
    2.加入判断，分开执行宁沪线和沪宁线
    3.将运行过程保存到日志
    4.多线程处理
    5.加入无限循环
    6.加入计时任务

In [ ]:
import requests
import json
import pandas as pd
import arrow
from itertools import combinations
import time
import schedule
import threading

class HighSpeed(object):
    #定义实例化后的初始参数
    def __init__(self,date,from_station,to_station,direction):
        self.date = date
        self.from_station = from_station
        self.to_station = to_station
        self.direction = direction
        
    def get_pd(self):
        try:
            headers={
            "Accept": "text/html, application/xhtml+xml, application/xml; q=0.9, */*; q=0.8",
            "Connection": "Keep-Alive",
            "Cookie": "JSESSIONID=CE1E0910299807B0B5D6CE1AE4B37318; RAIL_DEVICEID=KIGorQUspNUQqHtpHqI4qcn_7n8neHBX58xwMxWqPc26JQIimw-uxozqOZEOb_cizOpzFiNc04ZF_sK3raRaFjH3Ao-IO7KIEkB24NnWaD_4ME1YfS8KrNhz637Z2NQIjcjzvKENUq6HfF0Thy8UHfZpscWtyFJf; RAIL_EXPIRATION=1587115346591; route=6f50b51faa11b987e576cdb301e545c4; BIGipServerotn=1490616586.50210.0000; _jc_save_toStation=%u5170%u5DDE%2CLZJ; _jc_save_wfdc_flag=dc; _jc_save_toDate=2020-04-14; _jc_save_fromDate=2020-04-14; _jc_save_fromStation=%u897F%u5B89%2CXAY",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18362"
            }
            url = 'https://kyfw.12306.cn/otn/leftTicket/query?leftTicketDTO.train_date='\
            +self.date+'&leftTicketDTO.from_station='+self.from_station+'&leftTicketDTO.to_station='+self.to_station+'&purpose_codes=ADULT'
            text = requests.get(url,headers=headers).text
            return text
        except:
            log_file = open(r'../../file/subject/log/log{}.txt'.format(self.date),'a')
            log_file.write('error'+str(arrow.now().format('HH:mm'))+'\n')
            log_file.close()
            

    def to_csv(self):
        try:
            text = self.get_pd()
            js = json.loads(text)
            data = js['data']['result']
            dt_all = []
            for dt in data:
                dt_all.append(dt.split('|'))

            df_use = pd.DataFrame(dt_all)
            df_use = df_use[df_use[3].str.contains('D') | df_use[3].str.contains('G')]
            df_use = df_use[[2,3,6,7,8,9,22,26,30,31,32,33]]
            df_use = df_use.rename(columns={2:'编号',3:'车次',6:'发车站',7:'到达站',
                                        8:'发车时间',9:'到达时间',22:'其他',26:'无座',
                                        30:'二等座',31:'一等座',32:'商务特等座',33:'动卧'})
            df_use.replace('',0,inplace=True)
            tm = arrow.now().format('HH:mm')
            df_use['record_time'] = tm

            df_use.to_csv(r'../../file/subject/{}_data{}.csv'.format(self.direction,self.date),mode='a',header=False)
            
            log_file = open(r'../../file/subject/log/log{}.txt'.format(self.date),'a')
            log_file.write('info:{} {} to {} finished! \n'.format(str(arrow.now().format('HH:mm')),self.from_station,self.to_station))
            log_file.close()
            time.sleep(2)
        except:
            print('no data')


        print('I am working')
            

if __name__ == '__main__':
    def process_huning():
        date = arrow.now().format('YYYY-MM-DD')
        hu_ning = ['SHH', 'KNH', 'SZH', 'WXH', 'CZH', 'DYH', 'ZJH', 'NJH']
        com = combinations(hu_ning,2)
        for ls in com:
            hspeed = HighSpeed(date,ls[0],ls[1],'hu_ning')
            hspeed.to_csv()
    def process_ninghu():
        date = arrow.now().format('YYYY-MM-DD')
        ning_hu = ['SHH', 'KNH', 'SZH', 'WXH', 'CZH', 'DYH', 'ZJH', 'NJH'][::-1]
        com = combinations(ning_hu,2)
        for ls in com:
            hspeed = HighSpeed(date,ls[0],ls[1],'ning_hu')
            hspeed.to_csv() 
    def run_threaded(job_func):
        job_thread =  threading.Thread(target=job_func)
        job_thread.start()
    
    schedule.every(1).minutes.do(run_threaded,process_huning)
    schedule.every(1).minutes.do(run_threaded,process_ninghu)
    
    while True:
        schedule.run_pending()
        time.sleep(1)